<a href="https://colab.research.google.com/github/Eliandry/Neural-networks/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import re
import numpy as np
from keras.layers import Dense,LSTM,Input,Dropout,Embedding
from keras.models import Sequential
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences

In [2]:
with open('/content/positive','r') as f:
  positive=f.readlines()
  positive[0]=positive[0].replace('\ufeff','')

with open('/content/negative','r') as f:
  neg=f.readlines()
  neg[0]=neg[0].replace('\ufeff','')

texts=positive+neg

count_pos=len(positive)
count_neg=len(neg)
total_ln=count_neg+count_pos
print(total_ln)

189


In [3]:
max=800
tokenizer = Tokenizer(num_words=max, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»', lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts(texts)

dist=list(tokenizer.word_counts.items())

In [4]:
max_text_len=10
data=tokenizer.texts_to_sequences(texts)
data_pad=pad_sequences(data,maxlen=max_text_len)
print(data_pad)

[[  1 214  27 ...  37  13 216]
 [ 46  71   2 ... 218 219 220]
 [ 38 222  24 ... 225 226 116]
 ...
 [  0   0   0 ... 104  54  58]
 [  0   0   0 ... 210  34   1]
 [  0   0   0 ...  84  47 211]]


In [5]:
x=data_pad
y=np.array([[1,0]]*count_pos+[[0,1]]*count_neg)
print(x.shape,y.shape)

(189, 10) (189, 2)


In [ ]:
indeces=np.random.choice(x.shape[0],size=x.shape[0],replace=False)
X=x[indeces]
Y=y[indeces]
print(indeces)

In [9]:
model=Sequential()
model.add(Embedding(max,128,input_length=max_text_len))
model.add(LSTM(128,return_sequences=True))
model.add(LSTM(64))
model.add(Dense(2,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 128)           102400    
_________________________________________________________________
lstm (LSTM)                  (None, 10, 128)           131584    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 2)                 130       
Total params: 283,522
Trainable params: 283,522
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(0.0001))
history = model.fit(X, Y, batch_size=32, epochs=70)

In [38]:
reverse_word=dict(map(reversed,tokenizer.word_index.items()))
def seq_to_text(list_ind):
  words = [reverse_word.get(letter) for letter in list_ind]
  return(words)


t = "за все нужно платить".lower()
data = tokenizer.texts_to_sequences([t])
data_pad = pad_sequences(data, maxlen=max_text_len)
print(seq_to_text(data[0]) )

res = model.predict(data_pad)
print(res, np.argmax(res), sep='\n')


['за', 'все', 'нужно', 'платить']
[[0.00113778 0.99886227]]
1
